In [2]:
import psutil
import time
import csv
import subprocess
from datetime import datetime



In [3]:
OUTPUT_FILE = "system_metrics.csv"
DURATION = 60
INTERVAL = 1

In [4]:
def collect_metrics(label):
    with open(OUTPUT_FILE, 'a', newline='') as file:
        writer = csv.writer(file)
        for _ in range(int(DURATION / INTERVAL)):
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            cpu = psutil.cpu_percent()
            memory = psutil.virtual_memory().percent
            disk = psutil.disk_io_counters().read_bytes + psutil.disk_io_counters().write_bytes
            net = psutil.net_io_counters().bytes_sent + psutil.net_io_counters().bytes_recv
            writer.writerow([timestamp, cpu, memory, disk, net, label])
            time.sleep(INTERVAL)


In [5]:
def create_csv_if_not_exists():
    try:
        with open(OUTPUT_FILE, 'x', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['timestamp', 'cpu_percent', 'memory_percent', 'disk_io', 'network_io', 'label'])
    except FileExistsError:
        pass

In [6]:
def inject_cpu_anomaly():
    print("Injecting CPU anomaly...")
    return subprocess.Popen(["stress-ng", "--cpu", "4", "--timeout", str(DURATION)])

In [7]:
def inject_memory_anomaly():
    print("Injecting Memory anomaly...")
    return subprocess.Popen(["stress-ng", "--vm", "2", "--vm-bytes", "80%", "--timeout", str(DURATION)])

In [8]:
def inject_disk_anomaly():
    print("Injecting Disk anomaly...")
    return subprocess.Popen(["dd", "if=/dev/zero", "of=/tmp/disktest", "bs=1M", "count=10000"])

In [9]:
def inject_network_anomaly():
    print("Injecting Network anomaly (simulated with ping flood)...")
    return subprocess.Popen(["ping", "127.0.0.1", "-f"])

In [11]:
if __name__ == "__main__":
    create_csv_if_not_exists()


    print("Collecting NORMAL data...")
    collect_metrics(label=0)

    print("\nCollecting ANOMALOUS data with CPU + Memory overload...")
    cpu_proc = inject_cpu_anomaly()
    mem_proc = inject_memory_anomaly()

    collect_metrics(label=1)

    cpu_proc.terminate()
    mem_proc.terminate()
    print("\nData collection complete. CSV saved as:", OUTPUT_FILE)


Injecting CPU anomaly...
Injecting Memory anomaly...
stress-ng: info:  [34723] setting to a 1 min run per stressor
stress-ng: info:  [34724] setting to a 1 min run per stressor
stress-ng: info:  [34723] dispatching hogs: 4 cpu
stress-ng: info:  [34724] dispatching hogs: 2 vm
stress-ng: info:  [34723] note: system has only 133 MB of free memory and swap, recommend using --oom-avoid
stress-ng: info:  [34724] note: system has only 133 MB of free memory and swap, recommend using --oom-avoid

Data collection complete. CSV saved as:stress-ng: info:  [34723] skipped: 0
stress-ng: info:  [34723] passed: 4: cpu (4)
stress-ng: info:  [34723] failed: 0
stress-ng: info:  [34723] metrics untrustworthy: 0
stress-ng: info:  [34723] successful run completed in 59.87 secs
 system_metrics.csv
